In [2]:
# scraping a game's shots from understat.com

# import modules and packages
import requests # pega o URL
from bs4 import BeautifulSoup # permite dar parse na página HTML
import json # transforma os dados em JSON
import pandas as pd # transforma o JSON em dataframe

In [3]:
base_url = 'https://understat.com/match/'
match = str(input('Please enter the match id: ')) #14547
url = base_url + match

Please enter the match id: 14547


In [5]:
url

'https://understat.com/match/14547'

In [9]:
res = requests.get(url) # pega a página
soup = BeautifulSoup(res.content, 'lxml') # vai dar parse ao conteúdo da página
scripts = soup.find_all('script') # pega todos os scripts na página (inspect + find 'shotsdata' - é o que queremos)

#scripts

[<script>
 			var THEME = localStorage.getItem("theme") || 'DARK';
 			document.body.className = "theme-" + THEME.toLowerCase();
 		</script>, <script>
 	var shotsData 	= JSON.parse('\x7B\x22h\x22\x3A\x5B\x7B\x22id\x22\x3A\x22392313\x22,\x22minute\x22\x3A\x221\x22,\x22result\x22\x3A\x22MissedShots\x22,\x22X\x22\x3A\x220.7719999694824219\x22,\x22Y\x22\x3A\x220.46700000762939453\x22,\x22xG\x22\x3A\x220.020873909816145897\x22,\x22player\x22\x3A\x22Mario\x20Lemina\x22,\x22h_a\x22\x3A\x22h\x22,\x22player_id\x22\x3A\x221299\x22,\x22situation\x22\x3A\x22SetPiece\x22,\x22season\x22\x3A\x222020\x22,\x22shotType\x22\x3A\x22LeftFoot\x22,\x22match_id\x22\x3A\x2214547\x22,\x22h_team\x22\x3A\x22Fulham\x22,\x22a_team\x22\x3A\x22Liverpool\x22,\x22h_goals\x22\x3A\x221\x22,\x22a_goals\x22\x3A\x221\x22,\x22date\x22\x3A\x222020\x2D12\x2D13\x2016\x3A30\x3A00\x22,\x22player_assisted\x22\x3Anull,\x22lastAction\x22\x3A\x22None\x22\x7D,\x7B\x22id\x22\x3A\x22392314\x22,\x22minute\x22\x3A\x223\x22,\x22result\x22

In [11]:
# cada script tag é uma lista de items
# shots data vai ser o 2º item
strings = scripts[1].string

#strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22392313\\x22,\\x22minute\\x22\\x3A\\x221\\x22,\\x22result\\x22\\x3A\\x22MissedShots\\x22,\\x22X\\x22\\x3A\\x220.7719999694824219\\x22,\\x22Y\\x22\\x3A\\x220.46700000762939453\\x22,\\x22xG\\x22\\x3A\\x220.020873909816145897\\x22,\\x22player\\x22\\x3A\\x22Mario\\x20Lemina\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x221299\\x22,\\x22situation\\x22\\x3A\\x22SetPiece\\x22,\\x22season\\x22\\x3A\\x222020\\x22,\\x22shotType\\x22\\x3A\\x22LeftFoot\\x22,\\x22match_id\\x22\\x3A\\x2214547\\x22,\\x22h_team\\x22\\x3A\\x22Fulham\\x22,\\x22a_team\\x22\\x3A\\x22Liverpool\\x22,\\x22h_goals\\x22\\x3A\\x221\\x22,\\x22a_goals\\x22\\x3A\\x221\\x22,\\x22date\\x22\\x3A\\x222020\\x2D12\\x2D13\\x2016\\x3A30\\x3A00\\x22,\\x22player_assisted\\x22\\x3Anull,\\x22lastAction\\x22\\x3A\\x22None\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22392314\\x22,\\x22minute\\x22\\x3A\\x223\\x22,\\x22result\\x22\\x3A\\x22SavedShot\\x22

In [14]:
# limpar lixo para ficar apenas com JSON data
ind_start = strings.index("('")+2 # index para saber onde começar - 2 posições à frente do ('
ind_end = strings.index("')") # index para saber onde acabar - no ')

json_data = strings[ind_start:ind_end]
json_data = json_data.encode('utf8').decode('unicode_escape')

# converter string para JSON format
data = json.loads(json_data)

#data

{'h': [{'id': '392313',
   'minute': '1',
   'result': 'MissedShots',
   'X': '0.7719999694824219',
   'Y': '0.46700000762939453',
   'xG': '0.020873909816145897',
   'player': 'Mario Lemina',
   'h_a': 'h',
   'player_id': '1299',
   'situation': 'SetPiece',
   'season': '2020',
   'shotType': 'LeftFoot',
   'match_id': '14547',
   'h_team': 'Fulham',
   'a_team': 'Liverpool',
   'h_goals': '1',
   'a_goals': '1',
   'date': '2020-12-13 16:30:00',
   'player_assisted': None,
   'lastAction': 'None'},
  {'id': '392314',
   'minute': '3',
   'result': 'SavedShot',
   'X': '0.8930000305175781',
   'Y': '0.3060000038146973',
   'xG': '0.04911304637789726',
   'player': 'Ivan Cavaleiro',
   'h_a': 'h',
   'player_id': '3683',
   'situation': 'OpenPlay',
   'season': '2020',
   'shotType': 'RightFoot',
   'match_id': '14547',
   'h_team': 'Fulham',
   'a_team': 'Liverpool',
   'h_goals': '1',
   'a_goals': '1',
   'date': '2020-12-13 16:30:00',
   'player_assisted': 'Ruben Loftus-Cheek',
  

In [21]:
# converter JSON para dataframe
# correr um loop pelos home e away (2 elements json distintos) e meter em listas
x = []
y = []
xg = []
team = []
result = []

# pega subsets de home e away data
data_home = data['h']
data_away = data['a']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'xG':
            xg.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'xG':
            xg.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])

# criar a dataframe
col_names = ['x', 'y', 'xg', 'team', 'result']
df = pd.DataFrame([x,y,xg,team,result],index=col_names)

# está na horizontal, transpor para a vertical
df = df.T

df

,x,y,xg,team,result
0,0.7719999694824219,0.46700000762939453,0.020873909816145897,Fulham,MissedShots
1,0.8930000305175781,0.3060000038146973,0.04911304637789726,Fulham,SavedShot
2,0.93,0.6719999694824219,0.316528856754303,Fulham,SavedShot
3,0.815,0.6270000076293946,0.036229103803634644,Fulham,SavedShot
4,0.86,0.355,0.23231318593025208,Fulham,Goal
5,0.8819999694824219,0.3609999847412109,0.015041506849229336,Fulham,MissedShots
6,0.7730000305175782,0.5129999923706055,0.0437021479010582,Fulham,MissedShots
7,0.970999984741211,0.4520000076293945,0.6981164813041687,Fulham,SavedShot
8,0.8180000305175781,0.30100000381469727,0.0429905503988266,Fulham,BlockedShot
9,0.9290000152587891,0.5570000076293945,0.3006933331489563,Fulham,MissedShots
